In [ ]:
info = os.listdir('/Users/alexbraun/Downloads/data/arroway_wood/data')
info = filter(lambda x: re.search('diffuse', x), info)
info = filter(lambda x: re.search('_a_', x), info)
info = map(lambda x: re.search('.*_(.*)_diffuse', x).group(1), info)
info = sorted(info)
idwood = pd.read_csv('/Users/alexbraun/google_drive/code/projects/texture_classifier/data/identifying_wood_data.csv')

temp = list(set(list(chain(*map(lambda x: x.split('-'), info)))))
def has_keyword(item):
    output = []
    for regex in temp:
        found = re.search(regex, item, re.IGNORECASE)
        if found:
            output.append(found.group(0))
    if output:
        return ' '.join(output)
    return False
    
for col in ['common_name']:#, 'scientific_name']:
    new_col = 'found_' + col
    idwood[new_col] = None
    idwood[new_col] = idwood[col].apply(has_keyword)
    
x = idwood[(idwood.found_common_name != False)]# | (idwood.found_scientific_name != False)]

In [ ]:
sci = x[x.found_scientific_name != False].index
# x.found_scientific_name.tolist()

In [ ]:
mask = x.found_common_name.apply(lambda x: x not in ['fir', 'cedar', 'white', 'black', 'red'])
x = x[mask][['common_name', 'found_common_name']]

In [26]:
# %%snakeviz
%cd /Users/alexbraun/google_drive/code/projects/texture_classifier/python 
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import StratifiedShuffleSplit, ShuffleSplit
import pickle
import core.image_scanner
reload(core.image_scanner)
from core.image_scanner import ImageScanner

import core.utils
reload(core.utils)
from core.utils import *

# root = '/Volumes/abraun.backup.01/project folders/texture_classifier/data'
root = '/Users/alexbraun/Documents/data/texture_classifier/data/msgpack'
%cd /Users/alexbraun/Documents/data/texture_classifier/data/msgpack

# spec = ['texture', 'image_id', 'image_class', 'common_name', 'pass_', 'extension']
# info = get_series_info(root, spec)
info = pd.read_pickle('/Users/alexbraun/Documents/data/texture_classifier/data/info.pkl')
info = info[(info.image_class == 'a') & (info.pass_ == 'diffuse')]
# info = info.head()

patches = 1000

y = Series(info.index).apply(lambda x: [x]) * patches
y = list(chain(*y.tolist()))
y = Series(y)

params = {
            'aspect_ratio':     1,
            'min_size':         0.05,
            'max_size':         0.1,
            'patches':          patches,
            'patch_resolution': (100, 100),
            'rotation':         'random'
}

data = []
for file_ in info.fullpath:
    data.append(pd.read_msgpack(file_))
    
data = pd.concat(data, axis=0)
data['y'] = y

# index = data.index
# np.random.shuffle(index.tolist())
# data = data.ix[index]

# data = data.head(10000)

# X = data.drop('y', axis=1)
# y = data['y']

# index = data.index.tolist()
# np.random.shuffle(index)
# data.index = index
# X = np.array(data.ix[index])

# data.to_msgpack('../random_scan.100.100x100.05-10.random.msgpack')
# shuffle data
# index = data.index.tolist()
# np.random.shuffle(index)
# data = data.ix[index]

# with open('../random_scan.1000.100x100.05-10.random.pkl', 'w') as pkl:
#     pickle.dump(data, pkl)


/Users/alexbraun/google_drive/code/projects/texture_classifier/python
/Users/alexbraun/Documents/data/texture_classifier/data/msgpack


In [14]:
clf = RandomForestClassifier()
params = {
    'n_estimators':        [10],
#     'criterion':'          gini',
#     'max_depth':           None,
#     'min_samples_split':   2,
#     'min_samples_leaf':    1,
#     'max_features':'       'auto',
#     'max_leaf_nodes':      None,
#     'bootstrap':           True,
#     'oob_score':           False,
    'n_jobs':              [-1]
#     'random_state':        None,
#     'verbose':             0,
#     'min_density':         None,
#     'compute_importances': None
}
# train, test = [x for x in StratifiedShuffleSplit(y, n_iter=1, test_size=0.2)][0]
# train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2)
grid = GridSearchCV(clf, params)
# %time grid.fit(X.ix[test], y.ix[test])
# print(grid.best_score_)
# print(grid.best_estimator_.score(X.ix[test], y.ix[test]))

In [28]:
x = pd.read_msgpack('/Users/alexbraun/Documents/data/texture_classifier/data/random_scan.1000.100x100.05-10.random.msgpack')

In [34]:
x = data.to_hdf('/Users/alexbraun/Documents/data/texture_classifier/data/random_scan.1000.100x100.05-10.random.hdf', 'wood')

/Applications/anaconda/envs/dev/lib/python2.7/site-packages/pandas/io/pytables.py:270: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

  f(store)


In [35]:
x = pd.read_hdf('/Users/alexbraun/Documents/data/texture_classifier/data/random_scan.1000.100x100.05-10.random.hdf', 'wood')

In [37]:
from pandas.io.pytables import HDFStore

In [38]:
hdf = HDFStore('/Users/alexbraun/Documents/data/texture_classifier/data/random_scan.1000.100x100.05-10.random.hdf5')

In [39]:
hdf['info'] = info

In [44]:
hdf.close()

In [45]:
hdf = HDFStore('/Users/alexbraun/Documents/data/texture_classifier/data/random_scan.1000.100x100.05-10.random.hdf5', 'r')
hdf

<class 'pandas.io.pytables.HDFStore'>
File path: /Users/alexbraun/Documents/data/texture_classifier/data/random_scan.1000.100x100.05-10.random.hdf5
/info            frame        (shape->[1,7])